# Develop Model Driver

In this notebook, we will develop the API that will call our model. This module initializes the model, transforms the input so that it is in the appropriate format and defines the scoring method that will produce the predictions. The API will expect the input to be in JSON format. Once a request is received, the API will convert the json encoded request body into the image format. There are two main functions in the API: init() and run(). The init() function loads the model and returns a scoring function. The run() function process the images and uses the first function to score them.

    Note: Always make sure you don't have any lingering notebooks running (Shutdown previous notebooks). Otherwise it may cause GPU memory issue.

In [1]:
from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.image import Image
from azureml.core.model import Model
from dotenv import set_key, get_key, find_dotenv
import logging
from testing_utilities import img_url_to_json, get_auth

In [ ]:
import keras
import tensorflow
print("Keras: ", keras.__version__)
print("Tensorflow: ", tensorflow.__version__)

In [3]:
env_path = find_dotenv(raise_error_if_not_found=True)

## Write and save driver script

In [5]:
%%writefile driver.py

import tensorflow as tf
from resnet152 import ResNet152
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions

from azureml.core.model import Model

import numpy as np
import timeit as t
import base64
import json
from PIL import Image, ImageOps
from io import BytesIO
import logging

number_results = 3
logger = logging.getLogger("model_driver")

def _base64img_to_numpy(base64_img_string):
    decoded_img = base64.b64decode(base64_img_string)
    img_buffer = BytesIO(decoded_img)
    imageData = Image.open(img_buffer).convert("RGB")
    img = ImageOps.fit(imageData, (224, 224), Image.ANTIALIAS)
    img = image.img_to_array(img)
    return img

def create_scoring_func():
    """ Initialize ResNet 152 Model 
    """   
    start = t.default_timer()
    # model = ResNet152(weights='imagenet')
    model_name = 'resnet_model'
    model_path=Model.get_model_path(model_name)
    model = ResNet152()
    model.load_weights(model_path)
    end = t.default_timer()
    
    loadTimeMsg = "Model loading time: {0} ms".format(round((end-start)*1000, 2))
    logger.info(loadTimeMsg)
    
    def call_model(img_array):
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        preds = model.predict(img_array)
        preds = decode_predictions(preds, top=number_results)[0] 
        return preds
    
    return call_model       


def tuple_float_to_str(x):
        """tuple x = ('n02127052', 'lynx', 0.9816483) convert the 3rd element type numpy.float32 to string
           return: ('n02127052', 'lynx', '0.9816483')
        """
        a= list(x)
        a[2]= str(a[2])
        b= tuple(a)
        return b
    
def get_model_api():
    logger = logging.getLogger("model_driver")
    scoring_func = create_scoring_func()
    
    def process_and_score(inputString):
        """ Classify the input using the loaded model
        """
        start = t.default_timer()

        base64Dict = json.loads(inputString) 
        img_file_name, base64Img= next(iter(base64Dict.items())) 
        img_array = _base64img_to_numpy(base64Img)
        preds = scoring_func(img_array)
        
        # convert the 3rd element type of preds items from numpy.float32 to string
        for i, s in enumerate(preds):
            preds[i] = tuple_float_to_str(s)

        responses = {img_file_name: preds}
        
        end = t.default_timer()
        
        logger.info("Predictions: {0}".format(responses))
        logger.info("Predictions took {0} ms".format(round((end-start)*1000, 2)))
        return (responses, "Computed in {0} ms".format(round((end-start)*1000, 2)))
    return process_and_score

def init():
    """ Initialise the model and scoring function
    """
    global process_and_score
    process_and_score = get_model_api()
    
def run(raw_data):
    """ Make a prediction based on the data passed in using the preloaded model
    """
    return process_and_score(json.dumps(json.loads(raw_data)['input']))

Overwriting driver.py


## Test the driver¶ 
We test the driver by passing data.

In [6]:
logging.basicConfig(level=logging.DEBUG)

In [7]:
%run driver.py

Let's load the workspace.

In [ ]:
ws = Workspace.from_config(auth=get_auth())
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep="\n")

In [ ]:
model_path = Model.get_model_path('resnet_model', _workspace=ws)

In [10]:
IMAGEURL = "https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Lynx_lynx_poing.jpg/220px-Lynx_lynx_poing.jpg"

In [ ]:
# Always make sure you don't have any lingering notebooks running. Otherwise it may cause GPU memory issue.
init()

In [ ]:
jsonimg = img_url_to_json(IMAGEURL)
resp = run(jsonimg)

In [13]:
# Clear GPU memory
from keras import backend as K
K.clear_session()

Next, we will [build a docker image with this modle driver and other supporting files](03_BuildImage.ipynb).